# ArcPy Lab 1

Kuo Hong  
GEOG 458  
1/31/17



## Imports modules necessary for this lab

In [1]:
# Imports necessary modules for assignment
from subprocess import call
import os
import sys

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"

sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')

import arcpy
arcpy.env.overwriteOutput = True

In [2]:
# Change directory for the census block file and FIPS table.
block_file = "data/saep_bg10.shp"
table_file = "data/WashingtonFIPS.dbf"

## Pro-processing step

This code creates shapefiles for each county in the state, where each shapefile contains all block groups within that county.  This will allow me to convert these shapefiles to GeoJSON later.  Layer selection is used on the census block file by using the FIPS ID available in the .dbf.  Once all block groups with the same FIPS is selected, the CopyFeatures_management function can be used to create new shapefiles on those selections.  

This step took about an hour due to the ArcPy documentation not being correct with some syntax and having to troubleshoot.

In [3]:
# Makes new shapefiles for each individual county in the state. Takes in the block file and table file directory.
def make_county_shp(block_file, table_file):
    arcpy.MakeFeatureLayer_management(block_file, "blocks")
    cursor = arcpy.da.SearchCursor(table_file, ["CountyName", "FIPSCounty"])
    
    # For each FIPS, select all blocks with that FIPS and make a shapefile of the selection
    for row in cursor:
        query = "COUNTYFP10 = '" + row[1] + "'" # SQL query
        arcpy.SelectLayerByAttribute_management ("blocks", "NEW_SELECTION", query)
        arcpy.CopyFeatures_management("blocks", "data/" + row[0] + ".shp")

    del cursor

make_county_shp(block_file, table_file)

## Problem 1 - Conversion to GeoJSON

This code converts the shapefiles that I made in the previous step into GeoJSON using the code provided on Canvas.  The FIPS .dbf is searched through in order to get the input names, which are the names of the counties.  The output file will have the same name, but different file type, in the ```output``` folder.

This step also took about an hour of troubleshoot because some machines have ```OSGeo4W``` rather than ```OSGeo4W```, which took me a while to figure out.

In [4]:
# Converts the county shape files into GeoJSON files. Takes in the table file directory.
def convert(table_file):
    cursor = arcpy.da.SearchCursor(table_file, "CountyName")
    
    # Creates GeoJSON file from the county shapefiles.
    for row in cursor:
        input_name = "data/" + row[0] + ".shp"
        output_name = "output/" + row[0] + ".geojson" # To output to the output folder.
        call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
          '-f','GeoJSON','-t_srs','WGS84',
          '-s_srs','EPSG:2927',
          output_name,
          input_name])
    
    del cursor

convert(table_file)

## Problem 2 - Print top ten population

This code prints out the top ten population counts by county.  The census block file is iterated through row by row, where data is stored in a dictionary (key being the FIPS and value being the county population).  Once my dictionary is formed, I sorted it by descending value in order to print out the top ten county population.

This portion took about 30 minutes.

In [5]:
# Prints out the population count for the top ten counties with the most population.
def top10(block_file):
    pop = {}
    cursor = arcpy.da.SearchCursor(block_file, ["COUNTYFP10", "POP2017"])
    
    for row in cursor:
        if (row[0] not in pop):
            pop[row[0]] = row[1] # Adds new FIPS if it's not already in the dictionary
        else:
            pop[row[0]] += row[1] # Adds the block population to the amount that has already been added
    
    # Orders the population in descending order
    sorted_pop = sorted(pop.iteritems(), key=lambda (k,v): (v,k), reverse = True)
    
    # Prints top ten county populations
    print "Top 10 highest county population:"
    for i in range(10):
        print str(i + 1) + ". " + str(int(round(sorted_pop[i][1], 0)))
    
    del cursor

top10(block_file)

Top 10 highest county population:
1. 2153700
2. 859400
3. 789400
4. 499800
5. 471000
6. 276900
7. 264300
8. 253000
9. 216300
10. 193500
